In [2]:
import matplotlib.pyplot as plt
import cv2
import numpy as np
import pandas as pd
# from tabulate import tabulate

In [3]:
def read_and_resize(image_path, scale=1.0):
    image = cv2.imread(image_path)
    if scale != 1.0:
        image = cv2.resize(image, (0, 0), fx=scale, fy=scale)
    return image

def convert_to_grayscale(image):
    return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

def display_image(window_name, image):
    cv2.namedWindow(window_name, cv2.WINDOW_NORMAL)
    cv2.imshow(window_name, image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

### 4.1.1

In [4]:
# 4.1.1
I = read_and_resize('I.jpg')
J = read_and_resize('J.jpg')

I_gray = convert_to_grayscale(I)
J_gray = convert_to_grayscale(J)

display_image('I (Gray)', I_gray)
display_image('J (Gray)', J_gray)

diff = cv2.absdiff(I_gray, J_gray)
display_image('Różnica', diff)

### 4.2

In [ ]:
def calculate_optical_flow(I, J, block_size=7, search_range=3):
    """
        Oblicza przepływ optyczny pomiędzy dwoma obrazami w odcieniach szarości.
        
        :param I: Pierwszy obraz (wczesniejsza ramka).
        :param J: Drugi obraz (późniejsza ramka).
        :param block_size: Rozmiar bloku do analizy.
        :param search_range: Zakres przeszukiwania dla dopasowania bloku.
        :return: Macierze u i v zawierające wektory przepływu optycznego.
    """
    W2 = block_size // 2
    dX = dY = search_range
    height, width = I.shape
    u = np.zeros((height, width))
    v = np.zeros((height, width))

    for j in range(W2, height - W2):
        for i in range(W2, width - W2):
            IO = np.float32(I[j - W2:j + W2 + 1, i - W2:i + W2 + 1])
            min_dist = float('inf')
            for dy in range(-dY, dY + 1):
                for dx in range(-dX, dX + 1):
                    if 0 <= j + dy - W2 < height - W2 and 0 <= i + dx - W2 < width - W2:
                        JO = np.float32(J[j + dy - W2:j + dy + W2 + 1, i + dx - W2:i + dx + W2 + 1])
                        if IO.shape == JO.shape:
                            dist = np.sqrt(np.sum(np.square(JO - IO)))
                            if dist < min_dist:
                                min_dist = dist
                                u[j, i], v[j, i] = dx, dy

    return u, v

def visualize_optical_flow(I, u, v):
    """
        Wizualizuje przepływ optyczny na obrazie w formie mapy kolorów.
        
        :param I: Obraz bazowy.
        :param u: Macierz przesunięć w poziomie.
        :param v: Macierz przesunięć w pionie.
        :return: Obraz RGB z wizualizacją przepływu optycznego.
    """
    magnitude, angle = cv2.cartToPolar(u, v)
    hsv = np.zeros((I.shape[0], I.shape[1], 3), dtype=np.uint8)
    hsv[..., 0] = angle * 90 / np.pi
    hsv[..., 2] = 255
    hsv[..., 1] = cv2.normalize(magnitude, None, 0, 255, cv2.NORM_MINMAX)
    flow_rgb = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
    return flow_rgb

In [ ]:
I = cv2.imread('I.jpg', cv2.IMREAD_GRAYSCALE)
J = cv2.imread('J.jpg', cv2.IMREAD_GRAYSCALE)

cm1 = cv2.imread('cm1.png', cv2.IMREAD_GRAYSCALE)
cm2 = cv2.imread('cm2.png', cv2.IMREAD_GRAYSCALE)

u1, v1 = calculate_optical_flow(I, J)
flow_visualization1 = visualize_optical_flow(I, u1, v1)

u2, v2 = calculate_optical_flow(cm1, cm2)
flow_visualization2 = visualize_optical_flow(cm1, u2, v2)

cv2.imshow('Optical Flow Visualization (I)', flow_visualization1)
cv2.imshow('Optical Flow Visualization (Cm)', flow_visualization2)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
I = read_and_resize('I.jpg', 0.5)
J = read_and_resize('J.jpg', 0.5)

cm1 = read_and_resize('cm1.png', 0.5)
cm2 = read_and_resize('cm2.png', 0.5)

I_gray = convert_to_grayscale(I)
J_gray = convert_to_grayscale(J)

cm1_gray = convert_to_grayscale(cm1)
cm2_gray = convert_to_grayscale(cm2)

u1, v1 = calculate_optical_flow(I_gray, J_gray, block_size=5)
flow_visualization1 = visualize_optical_flow(I_gray, u1, v1)

u2, v2 = calculate_optical_flow(cm1_gray, cm2_gray, block_size=5)
flow_visualization2 = visualize_optical_flow(cm1_gray, u2, v2)

cv2.imshow('Optical Flow Visualization (I) with scale', flow_visualization1)
cv2.imshow('Optical Flow Visualization (cm) with scale', flow_visualization2)
cv2.waitKey(0)
cv2.destroyAllWindows()

### 4.3

In [ ]:
def of(I_org, I, J, W2=3, dY=3, dX=3):
    height, width = I.shape
    u = np.zeros((height, width))
    v = np.zeros((height, width))

    for j in range(W2, height - W2):
        for i in range(W2, width - W2):

            IO = np.float32(I[j - W2:j + W2 + 1, i - W2:i + W2 + 1])
            min_dist = float('inf')

            for dy in range(-dY, dY + 1):
                for dx in range(-dX, dX + 1):

                    if 0 <= j + dy - W2 < height - W2 and 0 <= i + dx - W2 < width - W2:
                        JO = np.float32(J[j + dy - W2:j + dy + W2 + 1, i + dx - W2:i + dx + W2 + 1])

                        if IO.shape == JO.shape:
                            dist = np.sqrt(np.sum(np.square(JO - IO)))

                            if dist < min_dist:
                                min_dist = dist
                                u[j, i], v[j, i] = dx, dy
    return u, v

def vis_flow(u, v, shape, name):
    magnitude, angle = cv2.cartToPolar(u, v)
    hsv = np.zeros((shape[0], shape[1], 3), dtype=np.uint8)
    hsv[..., 0] = angle * 90 / np.pi
    hsv[..., 2] = 255
    hsv[..., 1] = cv2.normalize(magnitude, None, 0, 255, cv2.NORM_MINMAX)
    flow_rgb = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
    cv2.imshow(name, flow_rgb)
    cv2.imwrite(name + '.png', flow_rgb)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

def pyramid(im, max_scale):
    images = [im]
    for k in range(1, max_scale):
        images.append(cv2.resize(images[k-1], (0,0), fx=0.5, fy=0.5))
    return images

def multi_scale_optical_flow(I_org, J_org, max_scale, W2=3, dY=3, dX=3):
    # Generowanie piramid obrazów
    IP = pyramid(I_org, max_scale)
    JP = pyramid(J_org, max_scale)
    
    # Inicjalizacja całkowitego przepływu optycznego
    total_u = np.zeros(I_org.shape)
    total_v = np.zeros(I_org.shape)

    # Przetwarzanie zaczynamy od najmniejszej skali
    I = IP[-1]
    for scale in range(max_scale-1, -1, -1):
        J = JP[scale]
        u, v = of(I_org, I, J, W2, dY, dX)  # Wyliczenie przepływu optycznego dla obecnej skali
        
        if scale > 0:
            # Modyfikacja I na podstawie przepływu, przygotowanie do następnej skali
            I_new = np.zeros_like(I)
            for y in range(I.shape[0]):
                for x in range(I.shape[1]):
                    newX = int(x + u[y, x])
                    newY = int(y + v[y, x])
                    if 0 <= newX < I.shape[1] and 0 <= newY < I.shape[0]:
                        I_new[y, x] = I[newY, newX]
            # Przygotowanie I do następnej skali
            I = cv2.resize(I_new, (0, 0), fx=2, fy=2, interpolation=cv2.INTER_LINEAR)
        # Sumowanie przepływów z różnych skal
        total_u += cv2.resize(u, (I_org.shape[1], I_org.shape[0]), interpolation=cv2.INTER_LINEAR)
        total_v += cv2.resize(v, (J_org.shape[1], J_org.shape[0]), interpolation=cv2.INTER_LINEAR)

    # Wizualizacja całkowitego przepływu optycznego
    vis_flow(total_u, total_v, I_org.shape, 'Total Optical Flow')


In [ ]:
I_org = cv2.imread('I.jpg', cv2.IMREAD_GRAYSCALE)
J_org = cv2.imread('J.jpg', cv2.IMREAD_GRAYSCALE)

multi_scale_optical_flow(I_org, J_org, 2)

In [ ]:
multi_scale_optical_flow(I_org, J_org, 2, 5, 5, 5)

In [ ]:
multi_scale_optical_flow(I_org, J_org, 3, 3, 3, 3)

In [ ]:
cm1 = cv2.imread('cm1.png', cv2.IMREAD_GRAYSCALE)
cm2 = cv2.imread('cm2.png', cv2.IMREAD_GRAYSCALE)

multi_scale_optical_flow(cm1, cm2, 2)

In [ ]:
multi_scale_optical_flow(cm1, cm2, 2, 5, 5, 5)

In [ ]:
multi_scale_optical_flow(I_org, J_org, 3, 3, 3, 3)

: 